# 7.6 Topology Optimization based on the Topological Derivative

We are again interested in minimizing a shape function of the type
\begin{align*}
    \mathcal J(\Omega) = \int_\Omega f(x) \, \mbox dx,\qquad f:\mathbb{R}^d\to \mathbb{R} 
\end{align*}
over all (admissible) shapes $\Omega \subset \mathsf{D}$ with $\mathsf{D}$ a given hold-all domain and $f \in  C(\overline{\mathsf{D}})$ a given function.

In [ ]:
# module to generate the 2D geometry
from netgen.geom2d import SplineGeometry


from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np

from interpolations import InterpolateLevelSetToElems

We represent the design $\Omega \subset \mathsf{D}$ by means of a level set function $\psi: \mathsf{D} \rightarrow \mathbb R$ in the following way:

\begin{align*}
    \psi(x) < 0 & \quad \Longleftrightarrow  \quad x \in \Omega \\
    \psi(x) = 0 & \quad \Longleftrightarrow  \quad x \in \partial \Omega \\
    \psi(x) > 0 & \quad \Longleftrightarrow  \quad x \in \mathsf{D} \setminus \overline \Omega.
\end{align*}

We define the domain $\mathsf{D} = [-2, 2]^2$:

In [ ]:
myMAXH = 0.05
geo = SplineGeometry()
R = 2

## add a rectangle
geo.AddRectangle(p1=(-R,-R),
                 p2=(R,R),
                 bc="rectangle",
                 leftdomain=1,
                 rightdomain=0)

mesh = Mesh(geo.GenerateMesh(maxh=myMAXH))
Draw(mesh)

We represent the level set function as a GridFunction on the mesh of $\mathsf{D}$.

In [ ]:
# H1-conforming finite element space
fes = H1(mesh, order=1) # C0 conforming finite element space
pwc = L2(mesh)   # piecewise constant (on each element) space

psi = GridFunction(fes)       # level set function
psi.vec[:]=1
psi_new = GridFunction(fes)   # grid function for line search

a = 4.0/5.0
b = 2
e = 0.001
f = CoefficientFunction( 0.1*( (sqrt((x - a)**2 + b * y**2) - 1) \
                * (sqrt((x + a)**2 + b * y**2) - 1) \
                * (sqrt(b * x**2 + (y - a)**2) - 1) \
                * (sqrt(b * x**2 + (y + a)**2) - 1) - e) )
Draw(f, mesh)
EPS = myMAXH * 1e-6      #variable defining when a point is on the interface and when not

In [ ]:
def Cost(psi):
    return IfPos(psi,0,f)*dx

print("initial cost = ", Integrate(Cost(psi), mesh))

The evolution of the level set function is guided by the $\textbf{topological derivative}$.
We define and draw the topological derivative in $\Omega$, $\mathsf{D}\setminus \Omega$ and the generalized topological derivative $g$.

In [ ]:
TDPosNeg_pwc = GridFunction(pwc)
TDNegPos_pwc = GridFunction(pwc)
TD_pwc = GridFunction(pwc)
CutRatio = GridFunction(pwc)

TD_node = GridFunction(fes)

scene1 = Draw(TD_node, mesh, "TD_node")
scene2 = Draw(TD_pwc, mesh, "TD_pwc")
scene3 = Draw(CutRatio, mesh, "CutRatio")


kappaMax = 1
kappa = 0.1*kappaMax

Redraw()

In define $NegPos$ and $PosNeg$ as the topological derivative in $\Omega$ respectively $\mathsf{D}\setminus\overline{\Omega}$:
\begin{align*}
    \mathsf{NegPos}(x) :=  -DJ(\Omega,\omega)(x) &= f(x) \quad \text{ for }x\in \Omega \\
    \mathsf{PosNeg}(x) :=   DJ(\Omega,\omega)(x) &= f(x) \quad \text{ for }x\in \overline{\mathsf{D}}\setminus\Omega^c
\end{align*}
and the level set update function (also called generalised topological derivative)
\begin{equation}
g_\Omega(x) := \chi_\Omega (x)\mathsf{NegPos}(x) + (1-\chi_\Omega(x))\mathsf{PosNeg}(x) = f(x). 
\end{equation}

In [ ]:
TDNegPos_pwc.Set(f)
TDPosNeg_pwc.Set(f)

InterpolateLevelSetToElems(psi, 1, 0, CutRatio, mesh, EPS)

# compute the combined topological derivative using the cut ratio information
for j in range(len(TD_pwc.vec)):
    TD_pwc.vec[j] = CutRatio.vec[j] * TDNegPos_pwc.vec[j] + (1-CutRatio.vec[j])*TDPosNeg_pwc.vec[j]
        
TD_node.Set(TD_pwc)

NormTD = sqrt(Integrate(TD_node**2*dx, mesh)) # L2 norm of TD_node

TD_node.vec.data = 1/NormTD * TD_node.vec        

scene1.Redraw()
scene2.Redraw()
scene3.Redraw()

The idea now is to make a fixed point iteration for $\ell\ge 0$
\begin{align*}
    \psi_{\ell+1} = (1-\kappa) \psi_\ell + \kappa \frac{g_{\psi_\ell}}{|g_{\psi_\ell}|_{L_2(\mathsf{D})}}, \qquad \kappa\in [0,1]
\end{align*}
Let us do one iteration:

In [ ]:
scene = Draw(psi, mesh)
kappaTest = 0.8
psi.vec.data =  (1-kappaTest)*psi.vec + kappaTest * TD_node.vec
scene.Redraw()

Now, let us repeat the procedure in the following iterative algorithm

In [ ]:
iter_max = 20
scene_psi = Draw(psi)
psi.Set(1)
scene_psi.Redraw()
psi_new.vec.data = psi.vec
kappa = 0.1*kappaMax
# input("Press enter to start optimization")
for k in range(iter_max):

        print("================ iteration ", k, "===================")

        # copy new levelset data from psi_new into psi
        psi.vec.data = psi_new.vec
        scene_psi.Redraw()
        
        # current cost function value
        J_current = Integrate(Cost(psi),mesh)

        print("cost in beginning of iteration", k, ": Cost = ", J_current)
        
        # level set update function
        TDPosNeg_pwc.Set(f)
        TDNegPos_pwc.Set(f)

        # get fraction of intersecting elements (for psi<0)
        InterpolateLevelSetToElems(psi, 1, 0, CutRatio, mesh, EPS)
        
        # weight the topological derivative with CutRatio
        for j in range(len(TD_pwc.vec)):
            TD_pwc.vec[j] = CutRatio.vec[j] * TDNegPos_pwc.vec[j] + (1-CutRatio.vec[j])*TDPosNeg_pwc.vec[j]
        
        TD_node.Set(TD_pwc)

        NormTD = sqrt(Integrate(TD_node**2*dx, mesh)) # L2 norm of TD_node
        TD_node.vec.data = 1/NormTD * TD_node.vec

        psi.vec.data  =  (1-kappa)*psi.vec + kappa * TD_node.vec

        for j in range(10): 

            # update the level set function
            psi_new.vec.data  =  (1-kappa)*psi.vec + kappa * TD_node.vec

            Jnew = Integrate(Cost(psi_new), mesh)
            
            Redraw()

            print("----------- Jnew in  iteration ", k, " = ", Jnew, "(kappa = ", kappa, ")")
            print('')

            if Jnew <= J_current+1e-10:
                print("----------------------------")
                print("======= step accepted ======")
                print("----------------------------")
                kappa = min(1.1*kappa, kappaMax)
                break
            else:
                kappa = kappa*0.8
        
        Redraw()
        print("iter" + str(k) + ", Jnew = " + str(Jnew) + "(kappa = ", kappa, ")")

        print("end of iter " + str(k))